In [ ]:
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torch.utils.data import Dataset
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Batch Size 
batch_size = 128
image_size = 64
workers = 2

numchannels = 3 #Number of channels 
latent = 100 #Latent Space Size 
gen_feat = 64 # Size of feature maps in generator 
dis_feat = 64 # Size of feature maps in discriminator 

# Number of gpus available. Use 0 for cpu.
num_gpu = 1 

In [ ]:
#Impliment data transforms and loading

#TODO
dataroot = 'Add Path' #Add your file path here
transforms = transforms.Compose() #Add Transforms

dataset = dset.ImageFolder(root=dataroot, transform=transforms)

# Create the dataloader
#TODO
dataloader = torch.utils.data.DataLoader()#Add dataloader

#Select device
device = torch.device("cuda:0" if (torch.cuda.is_available() and num_gpu > 0) else "cpu")

#Display real images
real_batch = next(iter(dataloader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

In [ ]:
#Weight initialization for Generator and Discriminator no changes required 
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
# Generator Code

class Generator(nn.Module):
    def __init__(self, num_gpu):
        super(Generator, self).__init__()
        self.num_gpu = num_gpu
        self.main = nn.Sequential(
            #Create Generator using ConvTranspose2d
            #TODO
            
            #One example layer
            nn.ConvTranspose2d(latent, gen_feat * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(gen_feat * 8),
            nn.ReLU(True),
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
# Create the generator
netG = Generator(num_gpu).to(device)

#Device selection
if (device.type == 'cuda') and (num_gpu > 1):
    netG = nn.DataParallel(netG, list(range(num_gpu)))

netG.apply(weights_init)

print(netG)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, num_gpu):
        super(Discriminator, self).__init__()
        self.num_gpu = num_gpu
        self.main = nn.Sequential(
            #Create Discriminator using Conv2d
            #TODO
            
            #One example layer
            nn.Conv2d(numchannels, dis_feat, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
# Create the Discriminator
netD = Discriminator(num_gpu).to(device)

#Device Selection
if (device.type == 'cuda') and (num_gpu > 1):
    netD = nn.DataParallel(netD, list(range(num_gpu)))

netD.apply(weights_init)

# Print the model
print(netD)

In [ ]:
#Select Loss Function
#TODO
criterion = 'None'

fixed_noise = torch.randn(64, latent, 1, 1, device=device)

#Real and fake labels for training
real_label = 1.
fake_label = 0.

# Setup optimizers for Generator and Discriminator 
#TODO
optimizerD = 'None'
optimizerG = 'None'

In [ ]:
# Training Parameters
num_epochs = 1 
lr = 0.0003
beta1 = 0.5

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader, 0):

        ## Train with all-real batch
        
        #Train Discriminator
        #TODO 


        # Generate fake images 
        #TODO


        #Train Generator
        #TODO

        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        #Record Losses to Plot
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1

In [ ]:
#Plot Your Generator and Discriminator Loss
#No Changes Needed
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
# Fake Images Grid 
#No Changes Needed
plt.figure(figsize=(15,15))
plt.subplot(1,1,1)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()